In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Project-2/Sport-Salaries/')


In [2]:
### Extract CSVs into DataFrames
NFL_file = "nfl_salary_2010_2020.csv"
NFL_df = pd.read_csv(NFL_file)
NFL_df.head()


,PLAYER,POS,TEAM,AGE,AVERAGE SALARY,TEAM AAV %,CASH EARNINGS,% OF TEAM PAYROLL,GTD EARNINGS,TEAM GTD %,CAP FIGURE,% OF TEAM CAP,Year
0,Tom Brady,QB,NE,33,18000000,0.17,16500000,0.2,-,-,20320000,-,2010
1,DeAngelo Hall,CB,WAS,26,9166667,0.06,18500000,0.14,-,-,19700000,-,2010
2,Peyton Manning,QB,IND,34,14171429,0.12,15800000,0.16,-,-,19266666,-,2010
3,Champ Bailey,CB,DEN,32,9000000,0.08,16500000,0.19,-,-,18895000,-,2010
4,Donovan McNabb,QB,WAS,33,14850000,0.1,18575000,0.14,-,-,18575000,-,2010


In [34]:
# Create a filtered dataframe from specific columns
nfl_cols = ["PLAYER", "POS", "TEAM", "AGE", "AVERAGE SALARY", "Year"]
nfl_transformed= NFL_df[nfl_cols].copy()

# Rename the column headers
nfl_transformed = nfl_transformed.rename(columns={"PLAYER": "player_name",
                                                          "POS": "position",
                                                          "TEAM": "team",
                                                          "AGE": "age",
                                                          "AVERAGE SALARY": "avg_salary",
                                                          "Year": "year"})

# Clean the data by dropping NA's, clean avg_salary, and setting the index
nfl_transformed = nfl_transformed.dropna(axis = 0, how ='any') 
nfl_transformed = nfl_transformed.sort_values('team', ascending=False).drop_duplicates(subset=['player_name', 'position', 'year'])

nfl_transformed_cond = nfl_transformed['avg_salary'] != "-"
nfl_transformed = nfl_transformed[nfl_transformed_cond]

nfl_transformed.set_index("year", inplace=True)
nfl_transformed.sort_values(by=['year'])
nfl_transformed.head()


,player_name,position,team,age,avg_salary
year,,,,,
2016,Colt McCoy,QB,WAS,29,3000000
2012,Domonique Johnson,CB,WAS,26,665000
2010,Rex Grossman,QB,WAS,30,755000
2020,Nick Sundberg,LS,WAS,33,1100000
2018,Adonis Alexander,CB,WAS,21,654048


In [17]:
#connect to postgres
connection_string = "postgres:akopova@localhost:5433/sports_db"
engine = create_engine(f'postgresql://{connection_string}')


In [36]:
#create NFL player table. Says there is an error, but table has been created.
pd.read_sql_query('CREATE TABLE nfl_player_salary(year INT, player_name VARCHAR(500), position VARCHAR(100), team VARCHAR(100), age INT, avg_salary INT, PRIMARY KEY (year, player_name, position))', con=engine).head()



ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [37]:
# Confirm table
engine.table_names()


['nfl_player_salary']

In [38]:
#Load into NFL player Salary
nfl_transformed.to_sql(name='nfl_player_salary', con=engine, if_exists='append', index=True)


In [42]:
pd.read_sql_query('select * from nfl_player_salary', con=engine).head()


,year,player_name,position,team,age,avg_salary
0,2016,Colt McCoy,QB,WAS,29,3000000
1,2012,Domonique Johnson,CB,WAS,26,665000
2,2010,Rex Grossman,QB,WAS,30,755000
3,2020,Nick Sundberg,LS,WAS,33,1100000
4,2018,Adonis Alexander,CB,WAS,21,654048
